In [5]:
import sys
from glob import glob
from os import path as osp
from datetime import datetime
from tqdm import tqdm
from skimage import io, transform
from matplotlib import pyplot as plt
import numpy as np
from math import sqrt
from statistics import mean
# torch imports
import torch
from torchvision import transforms
from torch.utils.data.dataloader import DataLoader

# root path of project
from os import path as osp
import sys

# get root directory
import re
reg = '^.*/AquaPose'
project_root = re.findall(reg, osp.dirname(osp.abspath(sys.argv[0])))[0]
sys.path.append(project_root)

from lib.dataset.PoseDataset import PoseDataset

from lib.models.keypoint_rcnn import get_resnet50_pretrained_model

# utils
from lib.utils.slack_notifications import slack_message
from lib.utils.select_gpu import select_best_gpu
from lib.utils.rmsd import kabsch_rmsd, kabsch_rotate, kabsch_weighted_rmsd, centroid, centroid_weighted, rmsd, rmsd_weighted

# optical vectors import
from lib.optical_vectors.optical_vectors import get_image_cut_coordinates, get_optical_vectors

# references import
# source: https://github.com/pytorch/vision/tree/master/references/detection
from references.engine import train_one_epoch, evaluate
from references.utils import collate_fn

from references.transforms import RandomHorizontalFlip


In [6]:
# load dataset to get a set of poses to match to
ref_dataset = PoseDataset([osp.join(project_root,'data/vzf/freestyle/freestyle_2'), osp.join(project_root,'data/vzf/freestyle/freestyle_3'), osp.join(project_root,'data/vzf/freestyle/freestyle_4'), osp.join(project_root,'data/vzf/freestyle/freestyle_6')], train=False)

test_dataset_dirs = [osp.join(project_root,'data/vzf/freestyle/freestyle_1')]

test_dataset = PoseDataset(test_dataset_dirs, train=False)

## Load csv file with displacements


In [17]:
dataset_root = test_dataset_dirs[0]
cut_dict = get_image_cut_coordinates(dataset_root)

print(cut_dict.keys())

dict_keys(['100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156', '157', '158', '159', '160', '161', '162', '163', '164', '165', '166', '167', '168', '169', '170', '171', '172', '173', '174', '175', '176', '177', '178', '179', '180', '181', '182', '183', '184', '185', '186', '187', '188', '189', '190', '191', '192', '193', '194', '195', '196', '197', '198', '199', '200', '201', '202', '203', '204', '205', '206', '207', '208', '209', '210', '211', '212', '213', '214', '215', '216', '217', '218', '219', '220', '221', '222', '223', '224', '225', '226', '227', '228', '229', '230', '231', '232', '233', '234', '235', '236', '237', '238', '239', '240', '2

In [20]:
test_id = 10
test_img, test_target = test_dataset[test_id]

img_id = test_target['image_id']
img_id  = str(img_id.item())
print(img_id)

opt_vec = get_optical_vectors(dataset_root, img_id, cut_dict)

130
tag 202021.25 width 5632 height 640


In [21]:
print(opt_vec)

[[[-0.02161958 -0.05319408]
  [ 0.00161332  0.00874542]
  [-0.01720057  0.01310228]
  ...
  [ 0.00628165 -0.06347029]
  [ 0.00709883 -0.07265346]
  [-0.03945971 -0.07523271]]

 [[-0.0317499   0.02315126]
  [-0.01962956  0.00085734]
  [-0.0006515   0.0142014 ]
  ...
  [-0.03082566 -0.02929589]
  [-0.03443485 -0.07003934]
  [ 0.01198218 -0.14346738]]

 [[-0.00049599  0.02314628]
  [ 0.00940565  0.0134732 ]
  [ 0.01587139  0.01633115]
  ...
  [-0.01984273 -0.04530816]
  [-0.00942983 -0.07438675]
  [ 0.03921162 -0.10981607]]

 ...

 [[ 0.05843601  0.04011784]
  [ 0.07769758  0.04781077]
  [ 0.08611163  0.01919252]
  ...
  [-0.02690808  0.06171544]
  [-0.0260918   0.05432007]
  [-0.04775034  0.09106293]]

 [[ 0.08179689  0.04892721]
  [ 0.0674668   0.03529525]
  [ 0.08228159  0.02647108]
  ...
  [-0.03062816  0.06400885]
  [-0.02638799  0.04972798]
  [-0.05199794  0.06200816]]

 [[ 0.05773206 -0.01768887]
  [ 0.03364699  0.01945345]
  [ 0.07136451  0.03551412]
  ...
  [-0.00184182  0.047704